# How have global energy production trends changed over time?

In [2]:
import pandas as pd
import numpy as np

## Goals

By the end of this case, you should be very comfortable writing your own functions using `pandas` and applying them to entire datasets. You'll understand how functions work in Python, including anonymous functions (using the keyword `lambda`), and you'll feel comfortable analyzing and manipulating larger datasets. You'll also have gained experience with exploring a dataset that is only loosely organised and about which you have very little initial information.


## Introduction

**Business Context.** Global electricity production, consumption, import, and export is complex and interesting for a variety of reasons. Each country has to keep track of a vast array of information to ensure that they produce enough electricity, yet balance these needs against medium-term financial implications and environmental concerns.

You are an analyst working at a non-governmental organization (NGO) that reports on global energy trends. Your department has acquired a large CSV file, but your colleagues are battling to extract relevant insights from it using Excel due to its size and format. Worse still, it has thousands of variables and they are not sure which ones are interesting. Thus, you have been made responsible for supporting your team's journalists by providing them with data and insights that they can turn into written reports.

**Business Problem.**  Your task is to **break the available data down into smaller files, understand the information that is available, and extract key insights for an upcoming report on global power patterns.** Specifically, your team wants you to answer the following questions:

* How much power is produced?
* How much power is consumed?
* How much power is imported and exported? 
* How much of this power is renewable?
* How are these trends in production, consumption, import, and export changing over time?

**Analytical Context.** The data is stored in a large CSV file containing information on power production and consumption by country and year. You will: 1) break down the data into summarized CSV files to share with your colleagues; 2) manipulate the data to create more categories from the existing columns; 3) find the biggest players in different categories, including total energy export and total production by type (e.g. nuclear); and finally 4) find trends in the data, such as which countries have the fastest growing energy production.

## Getting started with the International Energy Statistics data

The data file you have been given is a single CSV located at `data/all_energy_statistics.csv`. Your colleagues have informed you that the data is from http://data.un.org/Explorer.aspx, but they don't know much else about it. 

They specifically note that the data is very ["narrow"] (https://en.wikipedia.org/wiki/Wide_and_narrow_data). Although the file contains data for a wide variety of things, such as "Total Energy Production" all the way through to "Additives and Oxygenates - Exports", it has very few columns. 

Generally, when dealing with "wide" data, we can be fairly sure that all data in the same column is comparable. In this case, you'll notice a `unit` column. Not all numerical data in the `quantity` column is directly comparable. For example, sometimes the number in this column is defined in terms of "Metric tons, thousand" and sometimes in "Kilowatt-hours, million" -- evidently very different concepts!

As always, our first step is to read the data from disk and take a look at the first few rows:

In [3]:
df = pd.read_csv("all_energy_statistics.csv")

In [4]:
df

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.00,NaN,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.00,NaN,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.00,NaN,additives_and_oxygenates
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.00,NaN,additives_and_oxygenates
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.00,NaN,additives_and_oxygenates
5,Belgium,Additives and Oxygenates - Exports,2011,"Metric tons, thousand",25.00,NaN,additives_and_oxygenates
6,Belgium,Additives and Oxygenates - Exports,2010,"Metric tons, thousand",22.00,NaN,additives_and_oxygenates
7,Belgium,Additives and Oxygenates - Exports,2009,"Metric tons, thousand",45.00,NaN,additives_and_oxygenates
8,Czechia,Additives and Oxygenates - Exports,1998,"Metric tons, thousand",1.00,NaN,additives_and_oxygenates
9,Czechia,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",7.00,NaN,additives_and_oxygenates


You'll notice that there is more of a delay than before when running the `read_csv` function. This dataset has over 1 million rows, so it takes a while to load it all into memory. From the first rows, we can immediately gain some useful insights

* The `category` column looks like it is well organized. All the samples we see are lowercase and underscores are used instead of spaces
* The `commodity_transaction` column looks more like a human-readable description. We can see how it includes a description of the category (e.g. "additives_and_oxygenates" matches with "Additives and Oxygenates" and "wind_electricity" matches with "Electricity - ....wind....")
* We see `year` ranges from at least 1995 to 2014 
* As mentioned before, we'll need to be careful when comparing quantities, as the `unit` column might change the meaning of the `quantity` column.

A good first question to ask is how many unique values there are for the following columns:

* `country_or_area`
* `commodity_transaction`
* `year`
* `category`

Let's find out:

In [5]:
print(df.year.min())
print(df.year.max())
print("----------")
print("commodity_transaction")
print(df.commodity_transaction.unique())
print()
print("num unique values: ", len(df.commodity_transaction.unique()))
print()
print("----------")
print(df.category.unique())
print()
print("num unique values: ", len(df.category.unique()))
print()
print("---------------")
print(df.country_or_area.unique())
print()
print("num unique values: ", len(df.country_or_area.unique()))


1990
2014
----------
commodity_transaction
['Additives and Oxygenates - Exports' 'Additives and Oxygenates - Imports'
 'Additives and Oxygenates - Production' ...,
 'White spirit and special boiling point industrial spirits - Transformation'
 'White spirit and special boiling point industrial spirits - Transformation in petrochemical plants'
 'Electricity - total wind production']

num unique values:  2452

----------
['additives_and_oxygenates' 'animal_waste' 'anthracite' 'aviation_gasoline'
 'bagasse' 'biodiesel' 'biogases' 'biogasoline' 'bitumen' 'black_liquor'
 'blast_furnace_gas' 'brown_coal_briquettes' 'brown_coal' 'charcoal'
 'coal_tar' 'coke_oven_coke' 'coking_coal' 'conventional_crude_oil'
 'direct_use_of_geothermal_heat' 'direct_use_of_solar_thermal_heat'
 'electricity_net_installed_capacity_of_electric_power_plants' 'ethane'
 'falling_water' 'fuel_oil' 'fuelwood' 'gas_coke' 'gas_oil_diesel_oil'
 'gasoline_type_jet_fuel' 'gasworks_gas' 'geothermal' 'hard_coal' 'heat'
 'hydro'

We can see that `country_or_area` has 243 unique values, more than the officially recognised 195, because this list includes some former countries such as the USSR as well as areas like Antarctic Fisheries which are not formal countries.

As expected, the `categories` column is well standardized and breaks each row into one of 71 unique categories, while the `commodity_transaction` row is slightly more chaotic and consists of 2452 unique values.

In terms of time, our data ranges from 1990 - 2014 inclusive, so 25 years in total.

Note that the output of `unique()` is automatically truncated for large lists, with a `...` inserted to indicate this.

Since the `commodity_transaction` column is a bit chaotic, we'll need to touch it up a bit. Let's create a copy of our dataframe before we start changing it so we can refer back to the original values if necessary.

In [6]:
df_orig = df.copy()

The first thing we noticed about the `commodity_transaction` column is that it uses hyphens (`-`) as separators. We can also see that it uses lowercase and capital letters - often something that makes analysis harder if we are going to do any string matching (e.g. find the word "production", which might skip descriptions which use "Production" instead). 

Let's start by lowercasing all of the descriptions. In the previous case, you learned how to do this by creating a separate list, looping through the dataframe, and then adding all the items from the list as a new column. We could achieve what we wanted as follows:

In [7]:
%%time
clean_transaction_list = []

for item in df['commodity_transaction']:
    item = item.lower()
    clean_transaction_list.append(item)
    
df['clean_transaction'] = clean_transaction_list

CPU times: user 862 ms, sys: 77 ms, total: 939 ms
Wall time: 941 ms


In [8]:
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,clean_transaction
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,additives and oxygenates - exports


We added `%%time` at the top of our cell to make Jupyter output information about how long it took to run that cell. We can see that looping through our DataFrame and adding the column took nearly 1 second to complete. It also took 5 lines of code.

As its very common to need to apply the same operation on every row of a dataset, `pandas` provides a shortcut to do this. You can use the `.apply()` function on a DataFrame directly and pass in a function to apply to every row. This is more efficient in two ways:

* It takes fewer lines of code, so it's faster to write the code (and to read it)
* `apply()` is optimized to take advantage of modern CPU features such as vectorization, so it runs in less time

We can achieve exactly the same result as we did with our `for` loop using the `apply()` function as follows:

In [9]:
%%time
df['clean_transaction2'] = df['commodity_transaction'].apply(str.lower)

CPU times: user 482 ms, sys: 83.1 ms, total: 566 ms
Wall time: 558 ms


In [10]:
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,clean_transaction,clean_transaction2
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports


Here we can see that `.apply()` ran around twice as quickly as the iterative version and produced the same results (the `clean_transaction` and `clean_transaction2` columns are the same). You can read more about the `apply()` function [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html), but in essence you call it from a column of a DataFrame and pass in a function. It applies that function to every row of that column in the DataFrame. In this case, we passed in the `str.lower` function, which converts a string to lowercase.

## Pre-processing and pivoting our data

We noted before that the `commodity_transaction` column seemed to use hyphens to separate different concepts in a single column. Let's do some more analysis to see if this is true across the board.

### Exercise 1:

Find out how many of the 2,000+ unique columns contain:

- 0 hyphens
- exactly 1 hyphen
- more than 1 hyphen

**Hint:** You can use Python's built-in [`count()`](https://www.w3schools.com/python/ref_string_count.asp) method to count the occurrences of a character in a string).

**Answer.**

In [11]:
# Add code here
pd.Series(df["commodity_transaction"].unique()).str.count("-").value_counts()


1    1845
2     538
0      57
3      12
dtype: int64

In [12]:
pd.Series(df["commodity_transaction"].unique()).str.findall("([-])").str.len().value_counts()

1    1845
2     538
0      57
3      12
dtype: int64

In [13]:
hyphens_0 = 0
hyphens_1 = 0
hyphens_2plus = 0
for value in df.commodity_transaction.unique():
    hyphen_count = value.count("-")
    if hyphen_count == 0:
        hyphens_0 += 1
    elif value.count("-") == 1:
        hyphens_1 += 1
    else:
        hyphens_2plus += 1
print("zero hyphens", hyphens_0)
print("one hyphen", hyphens_1)
print("two or more hyphens", hyphens_2plus)

zero hyphens 57
one hyphen 1845
two or more hyphens 550


We can see that most descriptions have exactly one hyphen, strengthening the idea that the first part of the description before the hyphen is linked to `category`, while the rest is more descriptive. We should take a closer look at the ones with zero hyphens as there are only 57 of these.

### Exercise 2:

Write code to print out all descriptions with zero hyphens. What do you notice about these?

**Answer.**

In [14]:
df[df["commodity_transaction"].str.findall("([-])").str.len() == 0]["commodity_transaction"].unique()

array(['From chemical sources – Autoproducer',
       'From chemical sources – Autoproducer – CHP plants',
       'From chemical sources – Autoproducer – Heat plants',
       'From combustible fuels – Autoproducer',
       'From combustible fuels – Autoproducer – CHP plants',
       'From combustible fuels – Autoproducer – Heat plants',
       'From combustible fuels – Main activity',
       'From combustible fuels – Main activity – CHP plants',
       'From combustible fuels – Main activity – Heat plants',
       'From electric boilers – Main activity',
       'From heat pumps – Main activity',
       'From other sources – Autoproducer',
       'From other sources – Autoproducer – CHP plants',
       'From other sources – Autoproducer – Heat plants',
       'From other sources – Main activity',
       'From other sources – Main activity – CHP plants',
       'From other sources – Main activity – Heat plants',
       'Geothermal – Autoproducer',
       'Geothermal – Autoproducer – CHP 

In [15]:
for value in df.commodity_transaction.unique():
    hyphen_count = value.count("-")
    if hyphen_count == 0:
        print(value)

From chemical sources – Autoproducer
From chemical sources – Autoproducer – CHP plants
From chemical sources – Autoproducer – Heat plants
From combustible fuels – Autoproducer
From combustible fuels – Autoproducer – CHP plants
From combustible fuels – Autoproducer – Heat plants
From combustible fuels – Main activity
From combustible fuels – Main activity – CHP plants
From combustible fuels – Main activity – Heat plants
From electric boilers – Main activity
From heat pumps – Main activity
From other sources – Autoproducer
From other sources – Autoproducer – CHP plants
From other sources – Autoproducer – Heat plants
From other sources – Main activity
From other sources – Main activity – CHP plants
From other sources – Main activity – Heat plants
Geothermal – Autoproducer
Geothermal – Autoproducer – CHP plants
Geothermal – Autoproducer – Heat plants
Geothermal – Main activity
Geothermal – Main activity – CHP plants
Geothermal – Main activity – Heat plants
Nuclear – Main activity
Nuclear –

### Passing our own functions to `apply()`

We previously passed the built-in `str.lower()` function to the `apply()` function to apply to it every row in our DataFrame. Now we want to clean up the m-dashes and lowercase the result at the same time. Let's write our own custom Python function to do both, and pass that to `apply()` instead. You can read more about writing your own custom functions in Python [here](https://www.w3schools.com/python/python_functions.asp):

In [16]:
def clean_transaction_description(transaction_description):
    """Lowercase the input and replace all m-dashes with hyphens"""
    clean = transaction_description.lower()
    clean = clean.replace("–", "-")
    return clean
    

# drop the columns we added before so we can recreate them with our new clean function
df = df.drop(columns=['clean_transaction', 'clean_transaction2'])
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates


In [17]:
df['clean_transaction'] = df['commodity_transaction'].apply(clean_transaction_description)
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,clean_transaction
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,additives and oxygenates - exports


Here we used `apply()` again, but this time passed in our own function which did both the lowercasing and the replacing of m-dashes with hyphens.

We've now seen how to use the `apply()` function with both built-in functions and our own custom functions. There's one more way we can use `apply()` though: with custom **anonymous functions** using the Python `lambda` keyword. Let's see how to achieve the same result using `lambda`:

In [18]:
df = df.drop(columns=['clean_transaction'])
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates


In [19]:
# lowercase the description and replace m-dashes with hyphens in one line
df['clean_transaction'] = df['commodity_transaction'].apply(lambda x: x.lower().replace("–", "-"))
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,clean_transaction
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,additives and oxygenates - exports


This code is functionally equivalent to what we ran before, but it's more concise. Instead of giving our function a name (`clean_transaction_description`), we can declare an anonymous function by using the [`lambda`](https://www.w3schools.com/python/python_lambda.asp) keyword. This says that we are going to pass in a series of `x` values (the descriptions), and describes what to do to each of them. The advantage of doing this is that it's more concise. The disadvantage is that it can be harder to read and it prevents us from using our function again later without redefining it all over again.

### Extracting the most interesting rows

It's hard to manually inspect over 2,000 unique description values, but we know that we're mainly interested in: 

* Import
* Export
* Total production
* Total demand or consumption
* Renewables

We can search for some keywords in the descriptions using code similar to the following:

In [21]:
[x for x in df['clean_transaction'].unique() if "consumption" in x and "electricity" in x]

['sub-bituminous coal - consumption by electricity, chp and heat plants',
 'electricity - consumption by chemical and petrochemical',
 'electricity - consumption by commercial and public services',
 'electricity - consumption by construction',
 'electricity - consumption by food and tobacco',
 'electricity - consumption by households',
 'electricity - consumption by iron and steel',
 'electricity - consumption by machinery',
 'electricity - consumption by manufacturing, construction and non-fuel industry',
 'electricity - consumption by mining and quarrying',
 'electricity - consumption by non-ferrous metals',
 'electricity - consumption by non-metallic minerals',
 'electricity - consumption by other',
 'electricity - consumption by other manuf., const. and non-fuel ind.',
 'electricity - consumption by paper, pulp and print',
 'electricity - consumption by pipeline transport',
 'electricity - consumption by rail',
 'electricity - consumption by road',
 'electricity - consumption by te

This gives us a much more manageable list to look through, and we can see that "electricity - imports" is likely an interesting value. We can cross-check this in the main dataset (and see all columns to boot) as follows:

In [20]:
## Note the below is functionally equivalent to 
# df[df["clean_transaction"] == "electricity - imports"].head()
# but slightly easier to type

df[df.clean_transaction == "electricity - final energy consumption"].head(26)

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,clean_transaction
1091772,Afghanistan,Electricity - Final energy consumption,2014,"Kilowatt-hours, million",3767.3,1.0,total_electricity,electricity - final energy consumption
1091773,Afghanistan,Electricity - Final energy consumption,2013,"Kilowatt-hours, million",3022.0,NaN,total_electricity,electricity - final energy consumption
1091774,Afghanistan,Electricity - Final energy consumption,2012,"Kilowatt-hours, million",3265.0,NaN,total_electricity,electricity - final energy consumption
1091775,Afghanistan,Electricity - Final energy consumption,2011,"Kilowatt-hours, million",2300.0,NaN,total_electricity,electricity - final energy consumption
1091776,Afghanistan,Electricity - Final energy consumption,2010,"Kilowatt-hours, million",2400.0,NaN,total_electricity,electricity - final energy consumption
1091777,Afghanistan,Electricity - Final energy consumption,2009,"Kilowatt-hours, million",1600.0,1.0,total_electricity,electricity - final energy consumption
1091778,Afghanistan,Electricity - Final energy consumption,2008,"Kilowatt-hours, million",1500.0,1.0,total_electricity,electricity - final energy consumption
1091779,Afghanistan,Electricity - Final energy consumption,2007,"Kilowatt-hours, million",1500.0,1.0,total_electricity,electricity - final energy consumption
1091780,Afghanistan,Electricity - Final energy consumption,2006,"Kilowatt-hours, million",1240.0,1.0,total_electricity,electricity - final energy consumption
1091781,Afghanistan,Electricity - Final energy consumption,2005,"Kilowatt-hours, million",860.0,1.0,total_electricity,electricity - final energy consumption


In [21]:
df[df.clean_transaction == "electricity - gross demand"].head(26)

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,clean_transaction
1097271,Afghanistan,Electricity - Gross demand,2014,"Kilowatt-hours, million",4486.80,NaN,total_electricity,electricity - gross demand
1097272,Afghanistan,Electricity - Gross demand,2013,"Kilowatt-hours, million",4371.40,NaN,total_electricity,electricity - gross demand
1097273,Afghanistan,Electricity - Gross demand,2012,"Kilowatt-hours, million",3887.00,NaN,total_electricity,electricity - gross demand
1097274,Afghanistan,Electricity - Gross demand,2011,"Kilowatt-hours, million",3451.00,NaN,total_electricity,electricity - gross demand
1097275,Afghanistan,Electricity - Gross demand,2010,"Kilowatt-hours, million",2732.00,NaN,total_electricity,electricity - gross demand
1097276,Afghanistan,Electricity - Gross demand,2009,"Kilowatt-hours, million",1793.00,NaN,total_electricity,electricity - gross demand
1097277,Afghanistan,Electricity - Gross demand,2008,"Kilowatt-hours, million",1568.00,NaN,total_electricity,electricity - gross demand
1097278,Afghanistan,Electricity - Gross demand,2007,"Kilowatt-hours, million",1682.00,NaN,total_electricity,electricity - gross demand
1097279,Afghanistan,Electricity - Gross demand,2006,"Kilowatt-hours, million",1455.00,NaN,total_electricity,electricity - gross demand
1097280,Afghanistan,Electricity - Gross demand,2005,"Kilowatt-hours, million",993.00,NaN,total_electricity,electricity - gross demand


### Exercise 3:

Use the above method or any other method that you prefer to explore the transaction descriptions and define a Python list containing the 9 most interesting ones. These should cover the total values for import, export, total production, total demand, and renewable energy production.

**Answer.**

In [22]:
# The first four values handle demand, production, import and exports
# The others are all values that match the `total ..... production` pattern except for `thermal` which 
# loosely describes all non-renewable sources of production
keep_values =  ["Electricity - Gross demand",
        "Electricity - Gross production",
        "Electricity - imports",
        "Electricity - exports",
        "Electricity - total hydro production",
        "Electricity - total wind production",
        "Electricity - total solar production",
        "Electricity - total geothermal production",
        "Electricity - total tide, wave production",
               ]               
              

### Pivoting the interesting values into their own columns

Of course, now that we've identified the most interesting transaction descriptions, we probably ought to pull them out of that single column that they're stuck in. Let's "pivot" our data to a more useable format, keeping each of these interesting values as new columns. This translates our data from a fairly narrow format into a wider one.

You might know of "pivot tables" from Excel. If not, don't worry - you'll come across them later and in more detail. But if you do know of them, you'll recognize that this pretty much the exact same thing. We'll use the pivot function in pandas, which you can read more about [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html). For now, just try to understand how the following code works, but you won't be expected to do this yourself until you've gained more experience with `pandas`:

In [23]:
# we'll keep our "interesting" values after we turn them into columsn
# but we'll also keep the "country" and "year" columns
final_keep_values = ["country_or_area", "year"] + keep_values

# Turn values in the 'commodity transaction' column
# into our new column names
# and keep only the 'quantity' column as the new values
df_countries = pd.pivot_table(
    df,
    values="quantity",
    index=["country_or_area", "year"],
    columns="commodity_transaction",
).reset_index()[final_keep_values]

# rename the columns to be more concise
df_countries.columns = [
    "country",
    "year",
    "demand",
    "production",
    "imports",
    "exports",
    "hydro",
    "wind",
    "solar",
    "geothermal",
    "tide",
]

# output with the energy production leaders first
df_countries.sort_values(by="production", ascending=False)

,country,year,demand,production,imports,exports,hydro,wind,solar,geothermal,tide
1062,China,2014,5219096.0,5649583.4,6750.0,18158.0,1064337.0,156078.0,15189.0,NaN,NaN
1061,China,2013,5016127.0,5431637.4,7438.0,18669.0,920291.0,141197.0,5564.0,NaN,NaN
1060,China,2012,4609729.0,4987553.0,6874.0,17653.0,872107.0,95978.0,NaN,NaN,NaN
1059,China,2011,4319132.0,4713019.0,6562.0,19307.0,698945.0,70331.0,NaN,NaN,NaN
5322,United States,2010,4153664.0,4378422.0,45083.0,19107.0,286333.0,95148.0,3934.0,17577.0,NaN
5320,United States,2008,4153344.0,4368260.0,57019.0,24198.0,281995.0,55696.0,2091.0,16873.0,NaN
5319,United States,2007,4188972.0,4349840.0,51396.0,20143.0,275545.0,34603.0,1673.0,16798.0,NaN
5323,United States,2011,4142125.0,4349571.0,52301.0,15038.0,344679.0,120854.0,6153.0,17892.0,NaN
5326,United States,2014,4149377.0,4339210.0,66511.0,13298.0,281527.0,183892.0,24603.0,18710.0,NaN
5325,United States,2013,4128465.0,4306371.0,70355.0,11353.0,290113.0,169713.0,15872.0,18422.0,NaN


We can see thaht our data is in a much more user-friendly format now. We have kept only the quantity column and each row now represents one country in a particular year. If we had data for each year for each of the 243 countries or areas, we would expect to have 6075 rows, but we have only 5568. This makes sense as some countries stopped existing and data collection in general has become much easier and more consistent over time. Let's take a look at how many countries we have data on for each year:

In [24]:
df_countries['year'].value_counts()

2013    229
2014    229
2012    229
2007    227
2011    226
2008    226
2005    226
2009    226
2010    226
2006    226
2002    225
2004    225
2003    225
1995    223
1996    223
2001    223
1997    223
2000    222
1992    222
1993    222
1994    222
1998    222
1999    222
1990    200
1991    199
Name: year, dtype: int64

As expected, in earlier years, we have data for fewer countries.

The final check we should do is whether any of the values we kept used a different "unit". A quick scan of the data shows that all of the values we are interested in are measured in "Kilowatt-hours, million", but it's possible that some small values could be measured as "Kilowatt-hours, thousand", for example. Let's look for unique values used in our `keep_values` list:

In [25]:
x = keep_values[0]
all_units = []

for value in keep_values:
    units_used = list(df[df.commodity_transaction == value]['unit'].unique())
    all_units += units_used
print(set(all_units))

{'Kilowatt-hours, million'}


All good! Only one unit is used. So we are done with data preparation and we can start exploring our dataset for information.

## Exploring growth of power production and renewables

As mentioned, the team is interested in analyzing countries based on their renewable energy production. We currently know how much power they produce in total and how much of this is due to each of a number of renewable options. We'll start by adding some supplementary data and then analyzing our dataset for interesting countries and patterns.

### Exercise 4:

Add a new summary column called `renewable_percent` which gives the percentage of total power production which is made up of renewable energy.

**Hint:** You might notice that some values are `na`, meaning `not available`. We can probably assume that these are 0 (though this might not always be meaningful; e.g. if we don't have data on the USSR in 2014, it's not because its power plants are all turned off!). You can use the `pandas` [`fillna`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) method to replace `na` values with 0.

**Answer.**

In [26]:
# replace the `na` values with 0
df_countries = df_countries.fillna(0)
# sum all renewable energy production sources and divide by the total energy production
df_countries["renewable_percent"] = (
    df_countries["hydro"]
    + df_countries["wind"]
    + df_countries["solar"]
    + df_countries["geothermal"]
    + df_countries["tide"]
) / df_countries['production']
df_countries

,country,year,demand,production,imports,exports,hydro,wind,solar,geothermal,tide,renewable_percent
0,Afghanistan,1990,1055.0,1128.0,0.0,0.0,764.0,0.0,0.0,0.0,0.0,0.677305
1,Afghanistan,1991,945.0,1015.0,0.0,0.0,690.0,0.0,0.0,0.0,0.0,0.679803
2,Afghanistan,1992,789.0,703.0,131.0,0.0,478.0,0.0,0.0,0.0,0.0,0.679943
3,Afghanistan,1993,780.0,695.0,130.0,0.0,475.0,0.0,0.0,0.0,0.0,0.683453
4,Afghanistan,1994,770.0,687.0,128.0,0.0,472.0,0.0,0.0,0.0,0.0,0.687045
5,Afghanistan,1995,753.0,675.0,120.0,0.0,466.0,0.0,0.0,0.0,0.0,0.690370
6,Afghanistan,1996,743.0,675.0,110.0,0.0,475.0,0.0,0.0,0.0,0.0,0.703704
7,Afghanistan,1997,728.0,670.0,100.0,0.0,485.0,0.0,0.0,0.0,0.0,0.723881
8,Afghanistan,1998,719.0,665.0,95.0,0.0,495.0,0.0,0.0,0.0,0.0,0.744361
9,Afghanistan,1999,739.0,685.0,95.0,0.0,505.0,0.0,0.0,0.0,0.0,0.737226


### Exercise 5:

Considering only the most recent year that we have data for (2014), which 5 countries produced the largest proportion of their power through renewables, and which 5 countries produced the smallest proportion of their power through renewables?

**Hint:** You can use the [`sort_values`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html) method in `pandas` to sort a DataFrame by a specific column, either descending or ascending.

**Answer.**

In [27]:
df_countries[(df_countries["year"] == 2014)].sort_values(
    by="renewable_percent"
).head(5)

,country,year,demand,production,imports,exports,hydro,wind,solar,geothermal,tide,renewable_percent
5053,Trinidad and Tobago,2014,9531.00,9891.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3873,Palau,2014,73.70,79.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
962,Cayman Islands,2014,620.74,620.74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1012,Chad,2014,206.00,225.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3798,Oman,2014,28343.00,29128.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# filter the dataframe by year to get only 2014 and then sort by renewable percent and take the top 5
df_countries[(df_countries["year"] == 2014)].sort_values(
    by="renewable_percent", ascending=False
).head(5)

,country,year,demand,production,imports,exports,hydro,wind,solar,geothermal,tide,renewable_percent
2891,Lesotho,2014,783.48,515.20,271.20,2.92,515.20,0.0,0.0,0.0,0.0,1.000000
49,Albania,2014,7791.43,4724.43,3250.45,183.45,4724.43,0.0,0.0,0.0,0.0,1.000000
611,Bhutan,2014,2085.46,7003.86,187.37,4991.90,7003.36,0.0,0.0,0.0,0.0,0.999929
3948,Paraguay,2014,13432.00,55282.30,0.00,41400.10,55276.40,0.0,0.0,0.0,0.0,0.999893
2328,Iceland,2014,17475.00,18122.00,0.00,0.00,12873.00,8.0,0.0,5238.0,0.0,0.999834


### Question:

Why do you think we are seeing a lot of very small countries on both lists?

Very small countries are not particularly representative of the global renewable power situation, so your team asks you to restrict your analysis only to countries that produce a lot of power.

### Exercise 6:

Repeat the above analysis but only look at the countries in the top 10% of total power production.

**Hint:** You can filter a DataFrame with multiple conditions by using the `&` symbol; e.g.:

`df_countries[df_countries.year == 2014 & df_countries.wind > 0]` 

would give you a DataFrame of all countries in 2014 which had produced at least some wind power.

**Answer.**

In [29]:
threshold = df_countries["production"].quantile(0.9)
df_countries[
    (df_countries.production > threshold) & (df_countries.year == 2014)
].sort_values(by="renewable_percent", ascending=False).head(5)

,country,year,demand,production,imports,exports,hydro,wind,solar,geothermal,tide,renewable_percent
3773,Norway,2014,124139.0,142327.0,6347.0,21932.0,136636.0,2216.0,0.0,0.0,0.0,0.975584
712,Brazil,2014,615629.0,590541.0,33778.0,3.0,373439.0,12211.0,16.0,0.0,0.0,0.653072
937,Canada,2014,591137.0,656225.0,12808.0,58421.0,382574.0,22538.0,1756.0,0.0,16.0,0.620037
4844,Sweden,2014,132375.0,153662.0,13852.0,29475.0,63872.0,11234.0,47.0,0.0,0.0,0.489080
5474,Viet Nam,2014,141136.0,145730.0,2053.0,880.0,61480.0,300.0,0.0,0.0,0.0,0.423935


_________

Of course, your team is also interested in looking at change in renewable energy over time. Let's look at the top and bottom 5 countries where the percentage of renewable energy they produced in 2014 is **very different** from the percentage in 1990.

### Exercise 7:

Add a new column to your DataFrame which displays the difference in percentage renewable energy production between 2014 and 1990. Which are the top and bottom 5 countries? What do you notice about these countries? Perform this analysis both with all countries and again with only those in the 10% of total power production.

**Hint:** you can use the `pivot()` method again to create a DataFrame which has 1990 and 2014 as columns and `renewable_percent` as values to help with this by using the following code

```
renewable_change = pd.pivot_table(
    df_countries, values="renewable_percent", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]
```

**Answer.**

In [30]:
# get a DataFrame with only the 1990 and 2014 values kept, and as columns
renewable_change = pd.pivot_table(
    df_countries, values="renewable_percent", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]
# add the diff column to see the chnage
renewable_change["diff"] = renewable_change[2014] - renewable_change[1990]
renewable_change.sort_values(by="diff", ascending=False).head(5)
renewable_change.sort_values(by="diff").head(5)



year,country,1990,2014,diff
195,Sri Lanka,0.998413,0.377650,-0.620763
176,Rwanda,0.976608,0.390884,-0.585724
96,Honduras,0.912549,0.373227,-0.539322
204,Suriname,0.858600,0.364792,-0.493808
226,United Rep. of Tanzania,0.896869,0.419360,-0.477509


In [31]:
# get only the top producers and redo the analysis
threshold = df_countries.production.quantile(0.9)
df_countries_large = df_countries[df_countries.production > threshold]
renewable_change = pd.pivot_table(
    df_countries_large, values="renewable_percent", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]
renewable_change["diff"] = renewable_change[2014] - renewable_change[1990]
renewable_change.sort_values(by="diff", ascending=False).head(5)
renewable_change.sort_values(by="diff").head(5)

year,country,1990,2014,diff
2,Brazil,0.927691,0.653072,-0.274618
9,India,0.247679,0.123335,-0.124344
24,Sweden,0.498512,0.489080,-0.009432
13,Japan,0.115881,0.114409,-0.001472
3,Canada,0.615727,0.620037,0.004310


### Exercise 8:

Your team is also interested in countries which are producing a lot more power now than they were 25 years ago. What are the top and bottom 10 countries in terms of growth of:

* Total power
* Renewable power

Note that because many countries were producing zero or very little renewable energy in 1990, doing a basic growth calculation will show that many countries have "infinite" (represented as `inf` in `pandas`) growth. To avoid this, restrict your results to countries which produced at least 1,000 units of renewable power in 1990 for the renewable growth analysis and at least 1,000 units of total power for the total growth analysis.

**Hint:** Assuming you add a column called `renewable_total`, you can use the following pivots to generate tables similar to before for both renewable growth and total growth:

```
renewable_growth = pd.pivot_table(
    df_countries, values="renewable_total", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]
```

```
total_growth = pd.pivot_table(
    df_countries, values="production", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]
```

**Answer.**

In [32]:
df_countries["renewable_total"] = (df_countries["hydro"]
    + df_countries["wind"]
    + df_countries["solar"]
    + df_countries["geothermal"]
    + df_countries["tide"])

renewable_growth = pd.pivot_table(
    df_countries[df_countries.renewable_total >= 1], values="renewable_total", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]

total_growth = pd.pivot_table(
    df_countries[df_countries.production >= 1], values="production", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]

In [33]:
# renewable_growth.dropna(inplace=True)

In [48]:
renewable_growth["growth"] = (renewable_growth[2014] - renewable_growth[1990]) / renewable_growth[1990]
total_growth["growth"] = (total_growth[2014] - total_growth[1990]) / total_growth[1990]
renewable_growth.sort_values(by="growth",ascending=False).head() # Top 10
# renewable_growth.sort_values(by="growth",ascending=False).tail() # Bottom 10

year,country,1990,2014,growth,growthLabel
85,Israel,3.0,859.0,285.333333,Growing fast
51,Equatorial Guinea,2.0,197.0,97.500000,Growing
116,Mozambique,284.0,16360.0,56.605634,Growing
120,Netherlands,141.0,6694.0,46.475177,Growing
28,Cambodia,57.0,1855.0,31.543860,Growing


### Exercise 9:

Finally, your team wants an easy-to-read label for each country based on total growth. They have given you the following specification for how the countries should be labeled:

* zero or negative growth = "No growth"
* 1% -100% growth = "Growing"
* over 100% growth = "Growing fast"
* NaN (if the data from 1990 or 2014 is NaN) = "Not Applicable"

Calculate the label for each country, using the `apply()` method for efficiency. 

**Hint:** You can check if the value of variable `x` is Nan as follows:

```
import numpy as np
np.isnan(x)
```

**Answer.**

In [58]:
def getGrowthLabel(x):
    label = ""
    
    if x<= 1:
        label = "No growth"
    elif x>= 1 and x<= 100:
        label = "Growing"
    elif x > 100:
        label = "Growing fast" 
    else:
        label = "Not applicable"
    
    return label 

renewable_growth["growthLabel"] = renewable_growth["growth"].apply(getGrowthLabel)
total_growth["growthLabel"] = total_growth["growth"].apply(getGrowthLabel)

renewable_growth.sort_values(by="growth",ascending=False).head() # Top 10
# renewable_growth.sort_values(by="growth",ascending=False).tail() # Bottom 10

# total_growth.sort_values(by="growth",ascending=False).head() # Top 10
total_growth.sort_values(by="growth",ascending=False).iloc[120:] # Bottom 10


year,country,1990,2014,growth,growthLabel
140,Mongolia,3348.0,6725.000,1.008662,Growing
48,Cook Islands,16.0,31.800,0.987500,No growth
92,Guinea,518.0,1015.000,0.959459,No growth
14,Bahamas,950.0,1853.000,0.950526,No growth
45,Colombia,36166.0,70059.000,0.937151,No growth
39,Central African Rep.,95.0,183.000,0.926316,No growth
208,Syrian Arab Republic,11468.0,21726.000,0.894489,No growth
239,Zambia,7771.0,14452.000,0.859735,No growth
170,Portugal,28501.0,52802.000,0.852637,No growth
72,Fiji,467.0,859.300,0.840043,No growth


## Largest importers and exporters of energy

The final thing that your team wants to look into is imports and exports of energy by country.

### Exercise 10:

Your team wants to know:

* Which countries have imported and exported the most power in total
* Which countries have imported the largest percentage of their *demand* and exported the largest percentage of their *production*

Do the analysis for all countries *and* for only countries with total production in the top 10%.

**Answer.**

## Writing new country-specific summary data to disk

Your team is delighted that you've managed to make sense of the data and extract some insights. They want to explore the data themselves too, but all of their existing tools are designed to analyze data from only one country at a time. They have asked that you create separate CSV files for each country, using the country as the file name, with a maximum of 25 rows per file (one per year) and columns for imports, exports, etc.

To do this, we use the [`to_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) function on a given DataFrame to write it to a file. We create a new directory called "output_csvs" in our working directory so that we don't clutter up our workspace with 243 CSV files. Then we run the following code to write our data to disk:

In [35]:
import os

OUTPUT_DIRECTORY = "output_csvs"

if not os.path.exists(OUTPUT_DIRECTORY):
    os.makedirs(OUTPUT_DIRECTORY)

for country in df_countries['country'].unique():
    country_df = df_countries[df_countries.country == country].drop(columns='country')
    country_df.to_csv(f"{OUTPUT_DIRECTORY}/{country}.csv")

## Conclusions 

We saw a number of interesting trends in the global energy industry. Specifically, we saw that many countries are relying more and more on renewables, but that some of the countries with fast-growing demand are forced to turn to non-renewable sources to keep up.

We also noticed that contrary to our expectations of some countries being "net importers" and others being "net exporters" of power, many countries actually both import *and* export large amounts of power.

## Takeaways

In this case, we covered some more features of `pandas` and got more practice with the features we covered previously. Specifically we saw how to:

* Use the `apply()` method in `pandas` with build-in functions, custom functions, and anonymous functions
* Work with large datasets and explore these using basic string matching to find interesting columns, and reformat the results into more convenient formats
* Pivot between wide and narrow formats
* Plot basic line plots
* Break up a large dataset into smaller ones and write these back to disk

While you'll learn more advanced functionality than this in later cases, these basics will be used again and again, so keep coming back to this case as reference material as often as you need.